# svm_Guido subset_ main thesis project -ReHo

In [1]:
import pandas as pd
import os
import torch
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, ConcatDataset, TensorDataset
from torchvision import transforms
from sklearn.model_selection import KFold
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## start of classsification using Reho

In [2]:
control_base_path = '/dbstore/zmohaghegh/UKBiobank_subset/summary_measures_2018/ReHo_Control_guido_subset/ReHo_Normalised_z/'
mdd_base_path = '/dbstore/zmohaghegh/UKBiobank_subset/summary_measures_2018/ReHo_MDD_guido_subset/ReHo_Normalised_z/'

#define data path for control and MDD seprately

control_files_path = [control_base_path + f for f in os.listdir(control_base_path) if f.startswith('ReHo')]
mdd_files_path = [mdd_base_path + f for f in os.listdir(mdd_base_path) if f.startswith('ReHo')]
print(len(control_files_path), len(mdd_files_path))

406 288


### load data with nan 

In [3]:
#loading data seprately and set a label column for Control=0 and MDD=1

control_dataset_reho_zero = [nib.load(c).get_fdata() for c in control_files_path]
control_labels = np.zeros((406))
mdd_dataset_reho_zero = [nib.load(m).get_fdata() for m in mdd_files_path]
mdd_labels = np.ones((288))

##### define numpy array for doing the zero nan

In [4]:
control_reho_zero_nan2= control_dataset_reho_zero
mdd_reho_zero_nan2 = mdd_dataset_reho_zero

print(mdd_reho_zero_nan2[0].shape)
print(len(mdd_reho_zero_nan2))
#print(mdd_reho_zero_nan2[0])

mdd_reho_zero_nan3 = np.array(mdd_reho_zero_nan2)
control_reho_zero_nan3 =np.array(control_reho_zero_nan2)
print(control_reho_zero_nan2[0].shape)
print(len(control_reho_zero_nan2))

#control_reho_zero_nan3[0]

(91, 109, 91)
288
(91, 109, 91)
406


In [5]:
len(mdd_dataset_reho_zero)

288

In [6]:
len(control_dataset_reho_zero)

406

In [7]:
len(mdd_dataset_reho_zero[0])
len(control_dataset_reho_zero[0])

91

In [8]:
mdd_dataset_reho_zero[0].shape
control_dataset_reho_zero[0].shape

(91, 109, 91)

In [9]:
control_dataset_reho_zero[400][:,34,56]

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,  1.68043089,  1.56969142,  1.86985874,  1.86096191,
        1.93146718,  2.11725903,  2.55462217,  1.91065693,  0.50844282,
        0.05930695,  0.66814291,  1.64210677,  1.08213818,  0.25858074,
       -0.56684363, -0.7189495 , -1.00350475, -1.3447212 , -1.21441877,
       -1.15065515, -1.24084473, -0.9478718 , -0.31155097, -0.26281989,
       -0.07350048,  0.28912649,  0.87050122,  1.24497306,  1.28742945,
        1.01400137,  1.10193563,  1.00789368,  0.5122267 , -0.2069111 ,
       -0.59625113, -0.91880149, -1.0220412 , -0.95108175, -0.74096751,
       -0.69755989, -1.0146656 , -0.39677244, -0.46503615, -0.83524621,
       -0.38343135, -0.36499518, -0.55830753, -0.06751805,  1.04009187,
        1.75702274,  1.7453953 ,  0.46958524, -0.02432077,  0.23

In [10]:
mdd_dataset_reho_zero[200][:,34,56]

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,  1.61047339,  1.77807271,  1.7013253 ,  1.39898086,
        1.33440161,  1.58714914,  1.79297769,  1.71759427,  0.82062691,
       -0.28643134,  0.02608258,  1.44496298,  1.79902351,  0.89062357,
       -0.1175086 , -0.39969423, -0.91698033, -1.10948527, -1.04074872,
       -1.08609724, -1.21075296, -1.36051333, -1.39010167, -0.93246561,
        0.255503  ,  1.13637698,  1.01420772,  1.13140702,  1.07804942,
        0.97265822,  1.00109255,  2.12645841,  1.31568182,  1.10955155,
        1.09693825, -0.48187041, -1.00842035, -1.31914806, -0.64701831,
       -1.24585545, -1.60447073, -0.34008807,  0.080991  , -0.48761007,
       -0.69806671, -0.14730394,  0.27225262,  0.41571775,  0.33710229,
       -0.08484425, -0.77901256, -0.30997238,  0.58442497,  1.18

In [11]:
#example control
zero_nan_control_800= np.nan_to_num(control_dataset_reho_zero[400],copy=True)
zero_nan_control_800[:,34,56]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.68043089,  1.56969142,  1.86985874,  1.86096191,
        1.93146718,  2.11725903,  2.55462217,  1.91065693,  0.50844282,
        0.05930695,  0.66814291,  1.64210677,  1.08213818,  0.25858074,
       -0.56684363, -0.7189495 , -1.00350475, -1.3447212 , -1.21441877,
       -1.15065515, -1.24084473, -0.9478718 , -0.31155097, -0.26281989,
       -0.07350048,  0.28912649,  0.87050122,  1.24497306,  1.28742945,
        1.01400137,  1.10193563,  1.00789368,  0.5122267 , -0.2069111 ,
       -0.59625113, -0.91880149, -1.0220412 , -0.95108175, -0.74096751,
       -0.69755989, -1.0146656 , -0.39677244, -0.46503615, -0.83524621,
       -0.38343135, -0.36499518, -0.55830753, -0.06751805,  1.04009187,
        1.75702274,  1.7453953 ,  0.46958524, -0.02432077,  0.23

In [12]:
#example mdd
zero_nan_mdd_700= np.nan_to_num(mdd_dataset_reho_zero[200],copy=True)
zero_nan_mdd_700[:,34,56]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.61047339,  1.77807271,  1.7013253 ,  1.39898086,
        1.33440161,  1.58714914,  1.79297769,  1.71759427,  0.82062691,
       -0.28643134,  0.02608258,  1.44496298,  1.79902351,  0.89062357,
       -0.1175086 , -0.39969423, -0.91698033, -1.10948527, -1.04074872,
       -1.08609724, -1.21075296, -1.36051333, -1.39010167, -0.93246561,
        0.255503  ,  1.13637698,  1.01420772,  1.13140702,  1.07804942,
        0.97265822,  1.00109255,  2.12645841,  1.31568182,  1.10955155,
        1.09693825, -0.48187041, -1.00842035, -1.31914806, -0.64701831,
       -1.24585545, -1.60447073, -0.34008807,  0.080991  , -0.48761007,
       -0.69806671, -0.14730394,  0.27225262,  0.41571775,  0.33710229,
       -0.08484425, -0.77901256, -0.30997238,  0.58442497,  1.18

In [13]:
#zero_nan_control
print(control_reho_zero_nan3.shape)
print(control_reho_zero_nan3[0].shape)
#print(control_reho_zero_nan3[0])
print(type(control_reho_zero_nan3))


for i in range(len(control_dataset_reho_zero)):    
    control_reho_zero_nan3[i] =np.nan_to_num(control_dataset_reho_zero[i],copy=True)

(406, 91, 109, 91)
(91, 109, 91)
<class 'numpy.ndarray'>


In [14]:
print(control_reho_zero_nan3.shape)
print(control_reho_zero_nan3[0].shape)
#print(control_reho_zero_nan3[0]) ## mdd label for control=0
#print(control_reho_zero_nan3[310])
control_reho_zero_nan3[310][:,34,56]


(406, 91, 109, 91)
(91, 109, 91)


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.92880917,  1.65505862,  1.50674844,  2.09607053,
        2.05625844,  2.41848874,  2.81477833,  2.72659802,  0.56506944,
       -0.57554001, -0.57232052,  0.4258582 ,  1.86898804,  2.42637134,
        0.73444235, -1.2418406 , -1.55629468, -0.67085862, -0.38477191,
       -0.59492254, -0.74451315, -1.08082223, -1.03288448, -0.70927638,
       -0.61947405, -0.0504877 ,  1.20987523,  1.3354032 ,  1.1982826 ,
        0.79785603,  1.10096228, -0.05129792, -0.35614774, -0.5955022 ,
        0.29684177,  0.87079233,  0.94436699, -0.40263551, -0.84259462,
       -0.74756503, -0.91505301, -0.75655597, -1.34471273, -1.12159681,
       -0.88892096, -0.87314522, -0.3770186 ,  0.09685825,  0.41978529,
        1.64266217,  1.3973223 ,  1.64998627,  1.87235522,  1.99

In [15]:
#zero_nan_mdd

print(mdd_reho_zero_nan3.shape)
print(mdd_reho_zero_nan3[0].shape)
print(type(mdd_reho_zero_nan3))

for i in range(len(mdd_dataset_reho_zero)):    
    mdd_reho_zero_nan3[i]=np.nan_to_num(mdd_dataset_reho_zero[i],copy=True)

mdd_reho_zero_nan3.shape

(288, 91, 109, 91)
(91, 109, 91)
<class 'numpy.ndarray'>


(288, 91, 109, 91)

In [16]:
print(mdd_reho_zero_nan3.shape)

mdd_reho_zero_nan3.shape
print(mdd_reho_zero_nan3[0].shape)
#print(mdd_reho_zero_nan3[0])
mdd_reho_zero_nan3[210][:,34,56]

(288, 91, 109, 91)
(91, 109, 91)


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  2.92767024,  2.50129795,  2.15384102,  1.92867947,
        1.90241611,  1.88913727,  2.41310835,  2.88107395,  2.60863209,
        1.42518449,  0.89326882,  1.04972172,  1.66911662,  0.99317372,
       -0.22121689, -1.22684491, -1.33506155, -1.18181908, -1.06473887,
       -1.03707254, -1.29128063, -0.53458744, -0.60374779, -0.9098686 ,
       -0.37560141,  0.5662055 ,  2.12424588,  2.19857931,  2.39012671,
        2.47695422,  2.28949332,  2.27417636,  1.62085092,  1.04468524,
        0.75087368,  1.01845455, -0.08467761, -0.64575356, -0.44880834,
       -0.96765566, -0.43611035,  0.08036396, -0.40112367, -0.97603345,
       -1.16249979, -1.04867637, -0.2953583 ,  0.54806519,  1.85960984,
        0.9078818 ,  0.4021323 ,  0.65362918,  1.12080324,  0.81

### load and add 1 channel to dimension of data 'without nan'

In [17]:
resh=np.reshape(control_reho_zero_nan3[0], (1, 91, 109, 91))
resh.shape

resh2=control_reho_zero_nan3[0][np.newaxis,:,:,:]
resh2.shape
control_reho_zero_nan3.shape

(406, 91, 109, 91)

In [18]:
control_reho_4d_zero_nan = [c[np.newaxis,:,:,:] for c in control_reho_zero_nan3]
print(len(control_reho_4d_zero_nan))
control_reho_4d_zero_nan[0].shape

mdd_reho_4d_zero_nan = [m[np.newaxis,:,:,:] for m in mdd_reho_zero_nan3]
print(len(mdd_reho_4d_zero_nan))
mdd_reho_4d_zero_nan[0].shape

406
288


(1, 91, 109, 91)

In [19]:
print(mdd_reho_4d_zero_nan[200].shape)

(1, 91, 109, 91)


In [20]:
print(control_reho_4d_zero_nan[400].shape)

(1, 91, 109, 91)


In [21]:
len(mdd_reho_4d_zero_nan)

288

### concat data for control and MDD without nan

In [22]:
dataset_ukbb_reho=np.concatenate((control_reho_4d_zero_nan,mdd_reho_4d_zero_nan))
labels_ukbb_reho=np.concatenate((control_labels,mdd_labels))

In [23]:
len(dataset_ukbb_reho)

694

In [24]:
dataset_ukbb_reho[0].shape

(1, 91, 109, 91)

In [25]:
print(dataset_ukbb_reho[400])
print(dataset_ukbb_reho[500]) 

#problem solved : mdd_zero_nan label have problem : instead of 1 integer we have an array (91,109,91)

print(dataset_ukbb_reho[0])

dataset_ukbb_reho[500].shape

[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  ...

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 

(1, 91, 109, 91)

## linear SVM model_ CV _ReHo

In [84]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split


In [89]:
X_data=dataset_ukbb_reho
Y_label=labels_ukbb_reho

print(X_data.shape, Y_label.shape)

(694, 1, 91, 109, 91) (694,)


In [93]:
# Split the data into a training set and a test set
X_train, X_test,y_train, y_test = train_test_split(X_data, Y_label,  test_size=0.20, train_size=0.8)

In [94]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(555, 1, 91, 109, 91) (555,)
(139, 1, 91, 109, 91) (139,)


In [50]:
#! pip install svm

  Using cached svm-0.1.0.tar.gz (3.4 kB)
  Using cached xmltodict-0.12.0-py2.py3-none-any.whl (9.2 kB)
  Created wheel for svm: filename=svm-0.1.0-py3-none-any.whl size=3481 sha256=6020efce8647f274ada1ebc108d57e835cf235e54863957c84d3672a287ad8a4
  Stored in directory: /data/zmohaghegh/.cache/pip/wheels/f1/c9/85/1d14e69f68589a01cd3ed9d3a214f5c5b31f1846e9d7e35daf
Successfully built svm


### reshape MRI data

In [ ]:
X_ftrain = X_train.reshape(X_train.shape[0], -1).copy()
X_ftest= X_test.reshape(X_test.shape[0], -1).copy()

### model fitting

In [ ]:
classifier =SVC()
classifier.fit(X=X_ftrain, y=y_train)

In [ ]:
print(X_ftrain.shape, X_train.shape)
print(X_ftest.shape, X_test.shape)

### plotting

In [ ]:
# Plot confusion matrix for test data 
plot_confusion_matrix(classifier, X_ftest, y_test)
plt.show()

## Accuracy train and test

In [ ]:
y_predicted_train = classifier.predict(X_ftrain)
Accuracy_train= accuracy_score(y_train,y_predicted_train)

In [ ]:
print(f'accuracy train :{100* Accuracy_train}')

In [ ]:
y_predicted_test = classifier.predict(X_ftest)
Accuracy_test= accuracy_score(y_test,y_predicted_test)

In [ ]:
print(f'accuracy test: {100 * Accuracy_test}')

## SVM with cross validation 

In [99]:
X_f = X_data.reshape(X_data.shape[0], -1).copy()
SVM_classifier = SVC()
cv_results = cross_val_score(SVM_classifier, X_f, Y_label, cv=5)

In [100]:
acc_avg=100* np.mean(cv_results)
std_avg= 100*np.std(cv_results, ddof=1)

print('Average cross-validation accuracy for SVM UKBB ReHo:',acc_avg )
print('Std cross-validation accuracy :', std_avg )

Average cross-validation accuracy for SVM UKBB ReHo: 58.35783547075384
Std cross-validation accuracy : 0.1888452780497169


## SVM Tutorial

>>> import matplotlib.pyplot as plt  
>>> from sklearn.datasets import make_classification
>>> from sklearn.metrics import plot_confusion_matrix
>>> from sklearn.model_selection import train_test_split
>>> from sklearn.svm import SVC
>>> X, y = make_classification(random_state=0)
>>> X_train, X_test, y_train, y_test = train_test_split(
...         X, y, random_state=0)
>>> clf = SVC(random_state=0)
>>> clf.fit(X_train, y_train)
SVC(random_state=0)
>>> plot_confusion_matrix(clf, X_test, y_test)  
>>> plt.show()  

>>> #Plot

>>> recall = tpc / (n_gt + 1e-16)  # recall curve
>>> precision = tpc / (tpc + fpc)  # precision curve
>>> ax.plot(recall, precision)
>>> ax.set_xlabel('Recall')
>>> ax.set_ylabel('Precision')
>>> ax.set_xlim(0, 1.01)
>>> ax.set_ylim(0, 1.01)
>>> fig.tight_layout()